In [ ]:
import os
import json
import time

import httpx
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col

from dotenv import load_dotenv

load_dotenv()

In [ ]:
def write_json(data, filepath):
    with open(filepath, "w") as file:
        json.dump(data, file, ensure_ascii=False)

In [ ]:
path_output = "./output"
network = "ethereum"

In [ ]:
config_params = {
    "URL": os.getenv("SNOWFLAKE_URL"),
    "ACCOUNT": os.getenv("SNOWFLAKE_ACCOUNT"),
    "USER": os.getenv("SNOWFLAKE_USER"),
    "PASSWORD": os.getenv("SNOWFLAKE_PASSWORD"),
    "ROLE": os.getenv("SNOWFLAKE_ROLE"),
    "WAREHOUSE": os.getenv("SNOWFLAKE_WAREHOUSE"),
}

session = Session.builder.configs(config_params).create()

In [ ]:
df = (
    session.table("HSTNS_BLOCKCHAIN_DB.MAIN_DAP.COLLECTION_WHITELIST")
    .select(col("COLLECTION_ID"))
    .filter(col("NETWORK") == "polygon")
    .collect()
)
df

In [ ]:
[item["COLLECTION_ID"] for item in df]

In [ ]:
headers = {
    "accept": "*/*",
    "content-type": "application/json",
    "x-api-key": os.getenv("RESERVOIR_API_KEY"),
}

### Collection

In [ ]:
url_create_collection_set = "https://api-polygon.reservoir.tools/collections-sets/v1"
url_collections = "https://api-polygon.reservoir.tools/collections/v7"

In [ ]:
body = {"collections": df["COLLECTION_ID"].tolist()}

resp = httpx.post(url_create_collection_set, headers=headers, json=body)
collection_set_id = resp.json().get("collectionsSetId")
collection_set_id

In [ ]:
params = {
    "collectionsSetId": collection_set_id,
    "sortBy": "updatedAt",
    "limit": 1000,
}

resp = httpx.get(url_collections, headers=headers, params=params)
resp.json()

In [ ]:
collections = resp.json().get("collections")
collections

In [ ]:
len(collections)

In [ ]:
resp_collections = [item["id"] for item in collections]
resp_collections

In [ ]:
set(df["COLLECTION_ID"].tolist()) - set(resp_collections)

In [ ]:
set(resp_collections) - set(df["COLLECTION_ID"].tolist())

### Attribute

In [ ]:
url_attributes = (
    "https://api-polygon.reservoir.tools/collections/{collection}/attributes/all/v4"
)
path_attributes = os.path.join(path_output, "attribute")
output_attributes_name = "{network}_{collection_id}_attribute.json"


for collection_id in df["COLLECTION_ID"].tolist():
    resp = httpx.get(
        url_attributes.format(collection=collection_id), headers=headers, timeout=30
    ).json()

    resp["network"] = network
    resp["collection_id"] = collection_id

    write_json(
        resp,
        os.path.join(
            path_attributes,
            output_attributes_name.format(network=network, collection_id=collection_id),
        ),
    )

### Tokens

In [ ]:
network = "ethereum"
collection_id = "0x036721e5a769cc48b3189efbb9cce4471e8a48b1"
url_tokens = "https://api.reservoir.tools/tokens/v7"
path_tokens = os.path.join(path_output, "test")
output_tokens_name = "{network}_{collection_id}_{idx:06d}_token.json"

In [ ]:
idx = 0
continuation = ""
fault_count = 0

while True:
    params = {
        "collection": collection_id,
        "limit": 1000,
        "sortBy": "updatedAt",
        "includeAttributes": True,
    }

    if continuation:
        params["continuation"] = continuation

    resp = httpx.get(url_tokens, params=params, headers=headers, timeout=30)
    resp.raise_for_status()

    # 500번 에러일 떄 단순 요청 미스라 판단하고 다시 요청
    if resp.status_code == 500:
        if fault_count > 3:
            time.sleep(10)
            fault_count += 1
            continue
        elif fault_count > 6:
            print(f"Too many faults: {idx}, {network}, {collection_id}, {continuation}")
            break
        else:
            fault_count += 1
            continue
    # 429 에러일 떄 10초 대기 후 다시 요청
    elif resp.status_code == 429:
        time.sleep(10)
        continue
    else:
        resp = resp.json()
        fault_count = 0

    write_json(
        resp,
        os.path.join(
            path_tokens,
            output_tokens_name.format(
                network=network, collection_id=collection_id, idx=idx
            ),
        ),
    )

    print(
        f"idx: {idx}, Downloaded {len(resp['tokens'])} tokens, continuation: {resp['continuation']}"
    )
    if len(resp["tokens"]) < 1000 or resp["continuation"] is None:
        break

    idx += 1
    continuation = resp["continuation"]

118건 기준 6분 30초

In [ ]:
def get_token(network, collection_id, continuation):
    url_tokens = "https://api.reservoir.tools/tokens/v7"

    resp = httpx.get(url_tokens, params=params, headers=headers, timeout=30)
    resp.raise_for_status()

### Transaction

In [ ]:
network = "ethereum"
collection_id = "0x036721e5a769cc48b3189efbb9cce4471e8a48b1"
url_transaction = "https://api.reservoir.tools/collections/activity/v6"
path_transaction = os.path.join(path_output, "test")
output_transactions_name = "{network}_{collection_id}_{idx:06d}_transactions.json"

In [ ]:
idx = 0
continuation = ""
fault_count = 0

while True:
    params = {
        "collection": collection_id,
        "limit": 50,
        "types": ["sale", "transfer", "mint"],
    }

    if continuation:
        params["continuation"] = continuation

    print(params)

    resp = httpx.get(url_transaction, params=params, headers=headers, timeout=30)

    # 500번 에러일 떄 단순 요청 미스라 판단하고 다시 요청
    if resp.status_code == 500:
        if fault_count > 3:
            time.sleep(10)
            fault_count += 1
            continue
        elif fault_count > 6:
            print(f"Too many faults: {idx}, {network}, {collection_id}, {continuation}")
            break
        else:
            fault_count += 1
            continue
    # 429 에러일 떄 10초 대기 후 다시 요청
    elif resp.status_code == 429:
        time.sleep(10)
        continue
    else:
        resp = resp.json()
        fault_count = 0

    write_json(
        resp,
        os.path.join(
            path_transaction,
            output_transactions_name.format(
                network=network, collection_id=collection_id, idx=idx
            ),
        ),
    )

    print(
        f"idx: {idx}, Downloaded {len(resp['activities'])} activities, continuation: {resp['continuation']}"
    )
    if len(resp["activities"]) < 50 or resp["continuation"] is None:
        break

    idx += 1
    continuation = resp["continuation"]